In [33]:
from sklearn.linear_model import LinearRegression

In [34]:
linear_reg= LinearRegression()

In [43]:
test_encoded.head()

,id,Age,Height,Weight,Duration,Heart_Rate,Body_Temp,Calories,Sex_male
0,750000,45,177.0,81.0,7.0,87.0,39.8,0,True
1,750001,26,200.0,97.0,20.0,101.0,40.5,0,True
2,750002,29,188.0,85.0,16.0,102.0,40.4,0,False
3,750003,39,172.0,73.0,20.0,107.0,40.6,0,False
4,750004,30,173.0,67.0,16.0,94.0,40.5,0,False


In [36]:
y_train=train_encoded['Calories']

In [44]:
x_train=train_encoded.drop(['Calories'],axis=1)
test_encoded=test_encoded.drop(['Calories'],axis=1)

In [45]:
linear_reg.fit(x_train,y_train)

LinearRegression()

In [46]:
y_pred = linear_reg.predict(test_encoded)

In [48]:
y_pred

array([ 21.35250736, 114.61985896,  93.16064965, ...,  76.16885301,
       171.76592685,  88.378666  ])

In [49]:
y_pred = np.maximum(0, y_pred)  # clip negative predictions for safety

In [50]:
x_train.head()

,id,Age,Height,Weight,Duration,Heart_Rate,Body_Temp,Sex_male
0,0,36,189.0,82.0,26.0,101.0,41.0,True
1,1,64,163.0,60.0,8.0,85.0,39.7,False
2,2,51,161.0,64.0,7.0,84.0,39.8,False
3,3,20,192.0,90.0,25.0,105.0,40.7,True
4,4,38,166.0,61.0,25.0,102.0,40.6,False


In [51]:
submission = pd.DataFrame({
    'id': test_encoded['id'],        
    'Calories': y_pred
})

In [52]:
submission.to_csv('submission.csv', index=False)

In [53]:
from sklearn.ensemble import GradientBoostingRegressor

In [54]:
gbr=GradientBoostingRegressor()

In [55]:
gbr.fit(x_train,y_train)

GradientBoostingRegressor()

In [56]:
y_pred = gbr.predict(test_encoded)

In [ ]:
#hyperparameter tuning

gradient_params={"loss": ['squared_error'],
             "criterion": ['squared_error'],
             "min_samples_split": [2],
             "n_estimators": [100],
              "max_depth": [5, None]
}

In [61]:
from sklearn.model_selection import GridSearchCV

In [62]:
grid_search = GridSearchCV(
    estimator=gbr,
    param_grid=gradient_params,          
    cv=5,                        
    n_jobs=-1,                   
    verbose=2
)

grid_search.fit(x_train, y_train)

Fitting 5 folds for each of 8 candidates, totalling 40 fits


GridSearchCV(cv=5, estimator=GradientBoostingRegressor(), n_jobs=-1,
             param_grid={'criterion': ['friedman_mse', 'squared_error'],
                         'loss': ['squared_error'], 'max_depth': [5, None],
                         'min_samples_split': [2, 8], 'n_estimators': [100]},
             verbose=2)

In [63]:
best_model = grid_search.best_estimator_
best_params = grid_search.best_params_

In [66]:
grid_search.best_params_

{'criterion': 'friedman_mse',
 'loss': 'squared_error',
 'max_depth': 5,
 'min_samples_split': 2,
 'n_estimators': 100}

In [64]:
best_model = grid_search.best_estimator_
y_pred = best_model.predict(test_encoded)

In [68]:
y_pred = np.maximum(0, y_pred)


In [69]:
submission_gb = pd.DataFrame({
    'id': test_encoded['id'],     
    'Calories': y_pred
})
submission_gb.to_csv('submission_gb.csv', index=False)

In [71]:
import pickle

In [72]:
with open('linear_reg.pkl', 'wb') as f:
    pickle.dump(linear_reg, f)



In [73]:
with open('best_model.pkl', 'wb') as f:
    pickle.dump(best_model, f)
